In [305]:
import os
import yaml

from google.oauth2 import service_account
import os
import yaml
import git

## source vars

path = os.path.expanduser('~')

profile_pass = os.path.join(path,".droughty/profile.yaml")

with open(profile_pass) as f:
    lookml_config = yaml.load(f, Loader=yaml.FullLoader)

def get_git_root(path):

        git_repo = git.Repo(path, search_parent_directories=True)
        git_root = git_repo.git.rev_parse("--show-toplevel")
        return (git_root)
    
    
git_def_path = get_git_root(os.getcwd())

git_path = git_def_path

filename = 'droughty_project.yaml'

droughty_project = os.path.join(git_path,filename)

with open(droughty_project) as f:
    enviroment_project = yaml.load(f, Loader=yaml.FullLoader)



for key,value in enviroment_project.items():
    
    if key == 'profile':

        if lookml_config[value]['warehouse_name'] == 'big_query':

            if value in lookml_config:

        ## global vars 

                warehouse_name =  lookml_config[value]['warehouse_name']
                project_name =  lookml_config[value]['project_name']
                schema_name =  lookml_config[value]['schema_name']
                test_schemas = lookml_config[value]['test_schemas']
                
                warehouse_schema =   """

                with source as (

                    select * from `{0}.{1}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`

                    )

                    select * from source

                """.format(project_name,schema_name)

                dbml_reference_dict = """


                with source as (

                select * from `{0}.{1}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`

                ),

                    pks as (
                    select 
                    table_name as pk_table_name,
                    column_name as pk_column_name,
                    trim(column_name, "_pk") as pk_sk,
                    from source
                    where column_name like '%pk%'
                    ),

                    fks as (
                    select
                    table_name as fk_table_name,
                    column_name as fk_column_name,
                    trim(column_name, "_fk") as fk_sk,
                    from source
                    where column_name like '%fk%'

                    ),

                    references as (
                    select * from pks

                    inner join fks on pks.pk_sk = fks.fk_sk

                    )

                    select 
                    
                    *except (pk_column_name,pk_table_name),

                    case when pk_column_name is null
                        then 'not_available'
                    else pk_column_name
                    end as pk_column_name,

                    case when pk_table_name is null
                        then 'not_available'
                    else pk_table_name
                    end as pk_table_name                   
                    from source

                    left join references on source.column_name = references.fk_column_name and references.fk_table_name = source.table_name
                
                """.format(project_name,schema_name)

        elif lookml_config[value]['warehouse_name'] == 'snowflake':
            
            if value in lookml_config:

                warehouse_name =  lookml_config[value]['warehouse_name']
                project_name =  lookml_config[value]['project_name']
                schema_name =  lookml_config[value]['schema_name']
                test_schemas = lookml_config[value]['test_schemas']
                database = lookml_config[value]['database']

                snowflake_schema = '''

                select * from {0}.information_schema.columns;

                '''.format(database)

    ## warehouse test schemas

test_warehouse_schema =   """

        with source_1 as (

            select * from `{0}.{1}.INFORMATION_SCHEMA.COLUMNS`

            ),

        source_2 as (

        select * from `{0}.{2}.INFORMATION_SCHEMA.COLUMNS`
        
        ),
        
        source_3 as (

        select * from `{0}.{3}.INFORMATION_SCHEMA.COLUMNS`
        
        ),
        
        unioned as (

        select * from source_1
        
        union all
        
        select * from source_2
        
        union all
        
        select * from source_3
        
        )

        select * from unioned

""".format(project_name,test_schemas[0],test_schemas[1],test_schemas[2])

explores = (enviroment_project.get("explores"))

## 

explore_tables = []

for key,value in explores.items():

    for key,value in value.items():

        explore_tables.append(value)

## reduce explore_tables from array to single list
        
single_list_tables = [i[0] for i in explore_tables]


flat_list = []
for sublist in explore_tables:
    for item in sublist:
        flat_list.append(item)
        
final_list = []
for x in flat_list:
    final_list.append("'" + x + "'")
    
join_key_list = ['merge_counts_fk','merge_counts_pk']

params = {
    'project_id': project_name,
    'schema_id': schema_name, 
    'table_names': final_list,
    'table_names_unqouted': flat_list,
    'pk_fk_join_key_list': join_key_list
    
}


user_transaction_template = '''

with source as (

select 

*


from `{{project_id}}.{{schema_id}}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
where table_name in 
{{ table_names |inclause }}

),

{% for value in table_names_unqouted  %}

explore_table_row_count_{{ value | sqlsafe }} as (

select 

'{{ value | sqlsafe }}' as table_name,
count(*) as row_count

from `{{project_id}}.{{schema_id}}.{{ value | sqlsafe }}`

group by 1

),

{% endfor %}


merge_counts as (


{% for value in table_names_unqouted  %}

select * from explore_table_row_count_{{ value | sqlsafe }}

{% if not loop.last %}union all{% endif %}


{% endfor %}


),

pks as (
    select 
    table_name as pk_table_name,
    column_name as pk_column_name,
    trim(column_name, "_pk") as pk_sk,
    from source
    where column_name like '%%pk%%'
),

fks as (
    select
    table_name as fk_table_name,
    column_name as fk_column_name,
    trim(column_name, "_fk") as fk_sk,
    from source
    where column_name like '%%fk%%'

)


select 

'{{table_names_unqouted[0]}}' as parent_table_name,
pk_table_name,
pk_column_name,
fk_table_name,
fk_column_name,
merge_counts_pk.row_count as pk_row_count,
merge_counts_fk.row_count as fk_row_count,
merge_counts_parent.row_count as parent_row_count,

case when merge_counts_pk.row_count > merge_counts_fk.row_count
        then 'many_to_one'   
     when merge_counts_pk.row_count < merge_counts_fk.row_count
        then 'one_to_many'
     when merge_counts_pk.row_count = merge_counts_fk.row_count
        then 'one_to_one'
end as true_relationship,

case when merge_counts_pk.row_count < merge_counts_parent.row_count
    and merge_counts_fk.row_count < merge_counts_parent.row_count
     or  merge_counts_pk.row_count != merge_counts_parent.row_count
     and  merge_counts_fk.row_count != merge_counts_parent.row_count

        then 'many_to_one'
     when merge_counts_pk.row_count > merge_counts_parent.row_count
        then 'one_to_many'
     when merge_counts_fk.row_count > merge_counts_parent.row_count
        then 'one_to_many'
     when merge_counts_pk.row_count = merge_counts_parent.row_count
     or merge_counts_fk.row_count = merge_counts_parent.row_count
        then 'one_to_one'                
        
end as looker_relationship,

     
    
 
 
from pks

inner join fks on pks.pk_sk = fks.fk_sk

left join merge_counts as merge_counts_fk on merge_counts_fk.table_name = fks.fk_table_name

left join merge_counts as merge_counts_pk on merge_counts_pk.table_name = pks.pk_table_name

left join merge_counts as merge_counts_parent on merge_counts_parent.table_name = 'sales_applications'


order by looker_relationship


'''

In [306]:
final_list

["'sales_applications'",
 "'core_countries'",
 "'core_agents'",
 "'core_sponsors'",
 "'sales_application_agent_bridge'",
 "'core_users'",
 "'core_country_mapping'",
 "'core_institutions'",
 "'sales_student'",
 "'core_user_mapping'",
 "'core_program_intakes'",
 "'core_programs'",
 "'core_subjects'",
 "'sales_applications_summary'",
 "'product_progression_plans'",
 "'product_study_plans'"]

In [307]:
flat_list


['sales_applications',
 'core_countries',
 'core_agents',
 'core_sponsors',
 'sales_application_agent_bridge',
 'core_users',
 'core_country_mapping',
 'core_institutions',
 'sales_student',
 'core_user_mapping',
 'core_program_intakes',
 'core_programs',
 'core_subjects',
 'sales_applications_summary',
 'product_progression_plans',
 'product_study_plans']

In [308]:
from jinjasql import JinjaSql
j = JinjaSql(param_style='pyformat')
query, bind_params = j.prepare_query(user_transaction_template,params)

In [309]:
print(query % bind_params)



with source as (

select 

*


from `client-kaplan-poc.analytics_qa.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
where table_name in 
('sales_applications','core_countries','core_agents','core_sponsors','sales_application_agent_bridge','core_users','core_country_mapping','core_institutions','sales_student','core_user_mapping','core_program_intakes','core_programs','core_subjects','sales_applications_summary','product_progression_plans','product_study_plans')

),



explore_table_row_count_sales_applications as (

select 

'sales_applications' as table_name,
count(*) as row_count

from `client-kaplan-poc.analytics_qa.sales_applications`

group by 1

),



explore_table_row_count_core_countries as (

select 

'core_countries' as table_name,
count(*) as row_count

from `client-kaplan-poc.analytics_qa.core_countries`

group by 1

),



explore_table_row_count_core_agents as (

select 

'core_agents' as table_name,
count(*) as row_count

from `client-kaplan-poc.analytics_qa.core_agents`

group 

In [310]:
final_list

["'sales_applications'",
 "'core_countries'",
 "'core_agents'",
 "'core_sponsors'",
 "'sales_application_agent_bridge'",
 "'core_users'",
 "'core_country_mapping'",
 "'core_institutions'",
 "'sales_student'",
 "'core_user_mapping'",
 "'core_program_intakes'",
 "'core_programs'",
 "'core_subjects'",
 "'sales_applications_summary'",
 "'product_progression_plans'",
 "'product_study_plans'"]

In [103]:
final_list

["'sales_applications_fact'",
 "'wh_marketing_website_users_dim'",
 "'sales_quota_allocations'",
 "'core_users'",
 "'core_institutions'",
 "'sales_applications'",
 "'sales_student'",
 "'sales_application_summary'",
 "'core_countries'",
 "'core_agents'",
 "'core_sponsors'",
 "'product_study_plans'",
 "'sales_applications_fact'",
 "'quota_allocation_fk'",
 "'user_fk'",
 "'institution_fk'",
 "'application_fk'",
 "'student_fk'",
 "'country_fk'",
 "'agent_fk'",
 "'sponsor_fk'",
 "'study_plan_fk'"]

In [276]:
params = {
    'project_id': project_name,
    'schema_id': schema_name, 
    'table_names': final_list,
    'table_names_unqouted': flat_list,
    'pk_fk_join_key_list': join_key_list
    
}

In [277]:
parent_table = (explores.get("parent_table"))


In [278]:
parent_table

In [279]:
explores

{'sales_explore': {'parent_table': ['sales_applications'],
  'dimensions': ['core_countries',
   'core_agents',
   'core_sponsors',
   'sales_application_agent_bridge',
   'core_users',
   'core_country_mapping',
   'core_institutions',
   'sales_student',
   'core_user_mapping',
   'core_program_intakes',
   'core_programs',
   'core_subjects'],
  'facts': ['sales_applications_summary',
   'product_progression_plans',
   'product_study_plans']}}

In [27]:
def deep_get(dictionary, keys, default=None):
    return reduce(lambda d, key: d.get(key, default) if isinstance(d, dict) else default, keys.split("."), dictionary)

In [29]:
from functools import reduce

In [32]:
print (deep_get(enviroment_project, "explores.parent_table"))

None


In [94]:
parent_table = []

for key,value in explores.items():

    for key,value in value.items():

        parent_table.append(value[0])

In [95]:
parent_table[0]

'wh_marketing_website_event_tracks_fact'

In [137]:
table_names_unqouted[0]

NameError: name 'table_names_unqouted' is not defined

In [89]:
res = parent_table

In [53]:
print (enviroment_project.get('explores', {}).get('parent_table'))

None


In [51]:
enviroment_project

{'profile': 'ra-development',
 'explores': {'sales_explore': {'parent_table': ['wh_marketing_website_event_pages_fact'],
   'dimensions': ['wh_marketing_website_users_dim'],
   'facts': ['wh_marketing_website_event_tracks_fact']}}}

In [55]:
print(enviroment_project['explores']['parent_table'])

KeyError: 'parent_table'